In [25]:
import numpy as np
import pandas as pd
import os

In [26]:
DATASET_DIR = 'Kuzushiji-49'
DATASET_TRAIN_X_FILE = 'k49-train-imgs.npz'
DATASET_TRAIN_Y_FILE = 'k49-train-labels.npz'
DATASET_TEST_X_FILE = 'k49-test-imgs.npz'
DATASET_TEST_Y_FILE = 'k49-test-labels.npz'

In [27]:
train_images = np.load(os.path.join(os.getcwd(), DATASET_DIR, DATASET_TRAIN_X_FILE))['arr_0']
train_labels = np.load(os.path.join(os.getcwd(), DATASET_DIR, DATASET_TRAIN_Y_FILE))['arr_0']
test_images = np.load(os.path.join(os.getcwd(), DATASET_DIR, DATASET_TEST_X_FILE))['arr_0']
test_labels = np.load(os.path.join(os.getcwd(), DATASET_DIR, DATASET_TEST_Y_FILE))['arr_0']